# Advanced Analytics - Assignment 3

This assignment aims to predict the validity of Wikipedia Edits. The goal is to built a model which can classify incoming (stream) Wikipedia edits as Safe, Unsafe or Vandal. This notebook starts off by giving an overview of the contents. 

## Overview of the Notebook

> **1.** Problem Desciption <br>
> **2.** Data Capture & Cleaning <br>
> **3.** Text Procession & Ancillary Features <br>
> **4.** Model Training & Prediction <br>
> **5.** Model Evaluation <br>


## 1. Problem Description


## 2. Data Capture & Cleaning

## 3. Text Processing & Ancillary Features


### 3.1 Steps in the processing the text
> **1.** Read in Text & Compute difference between texts (python difflib)   
> **2.** Tokenization of words in difference in text <br>
> **3.** Normalization (lowercase), Stemming & Stop-Word Removal <br>
> **4.** Create Vector Representation
   > - (TF-IDF: https://spark.apache.org/docs/latest/mllib-feature-extraction.html#tf-idf)<br>
   > - (Word2Vec: )
    
> **5.** Ancillary Features (a.k.a. Metadata Features):
    > - Summary Comment 
    > - Revision Has Any Text (no empty revision)
    > - Vulgar words present -> vulgarism frequency and vulgarism impact
    > - Longest sequence of the same character in the edit (eg: boooooobs)
    > - Ratio of upper case letters to all letters?
    > - Similarity of old vs new text
    > - etc

## 4. Model Training & Prediction

### 4.1 (LogitBoost) Multinomial Bayesian Classifier